In [1]:
import pandas as pd

In [13]:
instruments = pd.read_csv("../data/kite/instruments.csv")
instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,258770438,1010822,EURINR24APRFUT,EURINR,0,2024-04-26,0.0,0.0025,1,FUT,BCD-FUT,BCD
1,262676230,1026079,EURINR24AUGFUT,EURINR,0,2024-08-28,0.0,0.0025,1,FUT,BCD-FUT,BCD
2,268449030,1048629,EURINR24DECFUT,EURINR,0,2024-12-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
3,256445190,1001739,EURINR24FEBFUT,EURINR,0,2024-02-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
4,267728646,1045815,EURINR24JANFUT,EURINR,0,2024-01-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
89617,2916865,11394,ZOTA,ZOTA HEALTH CARE,0,NaN,0.0,0.0500,1,EQ,NSE,NSE
89618,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0,NaN,0.0,0.0500,1,EQ,NSE,NSE
89619,979713,3827,ZUARIIND,ZUARI INDUSTRIES,0,NaN,0.0,0.0500,1,EQ,NSE,NSE
89620,2029825,7929,ZYDUSLIFE,ZYDUS LIFESCIENCES,0,NaN,0.0,0.0500,1,EQ,NSE,NSE


In [14]:
instruments = instruments.query('instrument_type == "EQ"')
instruments


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
8553,245304324,958220,001CSSL25,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8554,249407492,974248,001HCCL26,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8555,249406980,974246,001HCCL29,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8556,249407236,974247,001HCCL29A,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8557,249408004,974250,001HCCL29B,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
...,...,...,...,...,...,...,...,...,...,...,...,...
89617,2916865,11394,ZOTA,ZOTA HEALTH CARE,0,NaN,0.0,0.05,1,EQ,NSE,NSE
89618,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0,NaN,0.0,0.05,1,EQ,NSE,NSE
89619,979713,3827,ZUARIIND,ZUARI INDUSTRIES,0,NaN,0.0,0.05,1,EQ,NSE,NSE
89620,2029825,7929,ZYDUSLIFE,ZYDUS LIFESCIENCES,0,NaN,0.0,0.05,1,EQ,NSE,NSE


In [18]:
test = instruments.query('exchange != "NSE" and exchange !="BSE"')

In [19]:
test

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
20103,4105,16,CCIL-USDINR,CCILUSDINR,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20104,22281,87,COGEURINR,NaN,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20105,22793,89,COGGBPINR,NaN,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20106,23305,91,COGJPYINR,NaN,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20107,21769,85,COGUSDINR,NaN,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20108,20233,79,DUMMY1,11NSETEST,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20109,15625,61,DUSDINR,NaN,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20110,6409,25,EURINR,NaN,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS
20111,10505,41,EURUSD,NaN,0,NaN,0.0,0.0001,1,EQ,INDICES,CDS
20112,6665,26,GBPINR,NaN,0,NaN,0.0,0.0025,1,EQ,INDICES,CDS


In [20]:
instruments_bse = instruments.query('exchange == "BSE"')
instruments_bse

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
8553,245304324,958220,001CSSL25,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8554,249407492,974248,001HCCL26,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8555,249406980,974246,001HCCL29,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8556,249407236,974247,001HCCL29A,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
8557,249408004,974250,001HCCL29B,NaN,0,NaN,0.0,0.01,1,EQ,BSE,BSE
...,...,...,...,...,...,...,...,...,...,...,...,...
20098,286729,1120,SNSX50,BSE INDEX SNSX50,0,NaN,0.0,0.00,0,EQ,INDICES,BSE
20099,286985,1121,SNXT50,BSE INDEX SNXT50,0,NaN,0.0,0.00,0,EQ,INDICES,BSE
20100,287241,1122,TECK,BSE INDEX TECK,0,NaN,0.0,0.00,0,EQ,INDICES,BSE
20101,287497,1123,TELCOM,BSE INDEX TELCOM,0,NaN,0.0,0.00,0,EQ,INDICES,BSE


In [15]:
instruments_nse = instruments.query('exchange == "NSE"')
instruments_nse

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
20117,264713,1034,HANGSENG BEES-NAV,HANGSENG BEES-NAV,0,NaN,0.0,0.00,0,EQ,INDICES,NSE
20118,264969,1035,INDIA VIX,INDIA VIX,0,NaN,0.0,0.00,0,EQ,INDICES,NSE
20119,260617,1018,NIFTY 100,NIFTY 100,0,NaN,0.0,0.00,0,EQ,INDICES,NSE
20120,264457,1033,NIFTY 200,NIFTY 200,0,NaN,0.0,0.00,0,EQ,INDICES,NSE
20121,256265,1001,NIFTY 50,NIFTY 50,0,NaN,0.0,0.00,0,EQ,INDICES,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...
89617,2916865,11394,ZOTA,ZOTA HEALTH CARE,0,NaN,0.0,0.05,1,EQ,NSE,NSE
89618,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0,NaN,0.0,0.05,1,EQ,NSE,NSE
89619,979713,3827,ZUARIIND,ZUARI INDUSTRIES,0,NaN,0.0,0.05,1,EQ,NSE,NSE
89620,2029825,7929,ZYDUSLIFE,ZYDUS LIFESCIENCES,0,NaN,0.0,0.05,1,EQ,NSE,NSE


In [25]:
import requests

url = "https://api.upstox.com/v2/historical-candle/NSE_EQ|INE397D01024/1minute/2024-01-26/2024-01-01"

payload={}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"status":"success","data":{"candles":[["2024-01-25T15:29:00+05:30",1159.65,1162.25,1158,1159.75,16403,0],["2024-01-25T15:28:00+05:30",1161.8,1163.05,1157.05,1158.3,33047,0],["2024-01-25T15:27:00+05:30",1160.55,1163.9,1158.85,1162.05,54903,0],["2024-01-25T15:26:00+05:30",1160.85,1163.35,1158.95,1160.55,51491,0],["2024-01-25T15:25:00+05:30",1163.3,1163.35,1159.5,1160.85,49305,0],["2024-01-25T15:24:00+05:30",1163.1,1165.95,1162.4,1163.3,76826,0],["2024-01-25T15:23:00+05:30",1165.25,1166.35,1161.5,1163.45,127818,0],["2024-01-25T15:22:00+05:30",1165.25,1166.2,1164.2,1165.3,131696,0],["2024-01-25T15:21:00+05:30",1161,1165.8,1160.65,1165.1,149123,0],["2024-01-25T15:20:00+05:30",1160,1161.25,1159.55,1161,85425,0],["2024-01-25T15:19:00+05:30",1159.9,1161.55,1159.65,1159.95,127755,0],["2024-01-25T15:18:00+05:30",1159.25,1160.55,1158.3,1160,104008,0],["2024-01-25T15:17:00+05:30",1159,1159.75,1158.4,1159.25,42563,0],["2024-01-25T15:16:00+05:30",1160,1160.55,1158.45,1158.75,45220,0],["2024-01-25T1